# Packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Utils
import time
import os
loc = os.getcwd() 
import sys 

# Matplotlib Params
import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 16}

matplotlib.rc('font', **font)


# Tensorflow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score


# Data Wrangling

We need to import the data wrangle into a set of features and the output classes. We will try to predict the 6 different damage levels from the raw time series data itself (no processing required). 

In [4]:
data_dict = np.load('/Users/wang_to/Documents/University/Anomaly_detection/anomalies/bookshelf/dataset/data_dict.npy',allow_pickle='TRUE').item()
# change this to be relative to current repository directory

In [5]:
num_sens = 24
sens = []

for i in range(num_sens):
    sens.append(data_dict[f"Sensor{i+1}"])

sens = np.concatenate(sens)

sens_data = sens[:,:-4].astype('float32') # actual time series

sens = np.where(sens == 'D00', 0, sens) # find and replace: encode different damage levels
sens = np.where(sens == 'DB0', 1, sens)
sens = np.where(sens == 'DBB', 2, sens)
sens = np.where(sens == 'DHT', 3, sens)
sens = np.where(sens == 'D05', 4, sens)
sens = np.where(sens == 'D10', 5, sens)

sens_labels = sens[:, -4] # 0 and 1 labels for whether damaged or not
sens_damage_levels = sens[:, -2] # 0, 1, 2, 3, 4, 5 labels for different kinds of damage

In [6]:
print(f'Total data shape: {sens.shape}. Time series shape: {sens_data.shape}.') # 6480 = 24*270 experiments 

Total data shape: (6480, 8196). Time series shape: (6480, 8192).


We normalise the data below, although the network trains perfectly fine without the normalisation.

In [7]:
scaler = MinMaxScaler() # scale if required
scaler.fit(sens_data)
sens_data_normalised = scaler.transform(sens_data)

train_X = sens_data.reshape((-1, sens_data.shape[1], 1)) # training data, unnormalised
train_X_norm = sens_data_normalised.reshape((-1, sens_data_normalised.shape[1], 1)) # training data, normalised

In [8]:
data_damage_category = to_categorical(sens_damage_levels) # turn 0,1,2,3,4,5 into one-hot encoded vector

# Model, Training and Performance

## 1D Convolutional Network

### Model Parameters

We run the model over 7 epochs, using the ADAM optimisation algorithm with a learning rate of 1e-3 (note: lower values cause training to get stuck in a local minimum). We use 5-fold cross-validation and record a number of metrics. Each batch contains 128 training samples (we choose a larger amount of samples to have a good probability of containing samples from each class).

In [ ]:
num_epochs=7

optim = Adam(learning_rate=1e-3)

# 5-fold validation
num_folds = 5
kfold = KFold(n_splits = num_folds, shuffle=True, random_state=1337)
kfold.get_n_splits(train_X)
fold_no = 1 

scores_per_fold = []
predicts = []

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

# define model 
batch_size = 128
verbose = 1
input_shape = (batch_size, train_X.shape[1])

l1_reg = 1e-6
l2_reg = 1e-6

model_list = []

## Model Definition

The model itself is a 1D convolutional network, with a self-designed topology. Data is fed into an input convolutional layer, pooled, then fed through two "double-filters" (two-stacked convolutional layers) with pooling layers in-between, before being flattened and passed through a linear dense layer and then into the output. The filter sizes were chosen arbitrarily, although generally being powers of 2, and similarly with the number of filters, increasing as the depth of the network increased.

In [9]:
class book_conv1d_nn(tf.keras.Model):
    def __init__(self, n_outputs, type=False): # just get it working first. Set n_outputs = 1 for 1-class categorisation
        super(book_conv1d_nn, self).__init__()
        if type == 'heli':
            self.type = 'sigmoid'
        elif type == 'book':
            self.type = 'softmax'
        else: 
            print("Please provide a type argument.")
            return
        
        self.conv_in = Conv1D(
            filters=8,
            kernel_size=16, 
            input_shape=input_shape,
            activation='relu', 
            bias_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            activity_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), 
            padding='same'    

        )
        self.conv1 = Conv1D(
            filters=16, 
            kernel_size=8, 
            activation='relu',
            bias_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            activity_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), 
            padding='same', 
            name='hello'
        )
        self.conv2 = Conv1D(
            filters=32,
            kernel_size=4,
            activation='relu',
            bias_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            activity_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), 
            padding='same'   
        )

        self.conv3 = Conv1D(
            filters=32, 
            kernel_size=8, 
            activation='relu',
            bias_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            activity_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), 
            padding='same', 
            name='hello'
        )

        self.conv4 = Conv1D(
            filters=48, 
            kernel_size=8, 
            activation='relu',
            bias_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            activity_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), 
            padding='same', 
            name='hello'
        )

        self.maxPool = MaxPooling1D(pool_size=2, strides=2)
        self.flat = Flatten()
        self.D2 = Dense(n_outputs, activation = self.type)

    def call(self, inputs, training=False):
        x = self.conv_in(inputs)
        x = self.maxPool(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxPool(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.maxPool(x)
        x = self.flat(x)
        x = self.D2(x)
        return(x)


2022-02-22 16:51:56.343030: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-22 16:51:56.343602: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 10. Tune using inter_op_parallelism_threads for best performance.


Fold no: 1
-------------------------------------------
Fitting model on X_train, Y_train:
Train on 5184 samples
Epoch 1/7
5184/5184 [==============================] - 193s 37ms/sample - loss: 1.2857 - tp: 1925.0000 - fp: 1119.0000 - tn: 24801.0000 - fn: 3259.0000 - categorical_accuracy: 0.5538 - precision: 0.6324 - recall: 0.3713 - auc: 0.8302 - prc: 0.5190
Epoch 2/7
5184/5184 [==============================] - 194s 37ms/sample - loss: 1.0604 - tp: 2453.0000 - fp: 824.0000 - tn: 25096.0000 - fn: 2731.0000 - categorical_accuracy: 0.6233 - precision: 0.7486 - recall: 0.4732 - auc: 0.8887 - prc: 0.6632
Epoch 3/7
5184/5184 [==============================] - 202s 39ms/sample - loss: 0.7654 - tp: 3280.0000 - fp: 750.0000 - tn: 25170.0000 - fn: 1904.0000 - categorical_accuracy: 0.7342 - precision: 0.8139 - recall: 0.6327 - auc: 0.9439 - prc: 0.8076
Epoch 4/7
5184/5184 [==============================] - 196s 38ms/sample - loss: 0.4861 - tp: 4133.0000 - fp: 536.0000 - tn: 25384.0000 - fn: 1051.

## Training Loop

In [48]:
Y_test

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
for train_index, test_index in kfold.split(train_X):
    print(f'Fold no: {fold_no}')
    X_train, X_test = train_X[train_index], train_X[test_index]
    Y_train, Y_test = data_damage_category[train_index], data_damage_category[test_index]

    model = book_conv1d_nn(n_outputs=6, type='book') # creates a new model every loop

    model.compile(optimizer='adam', loss='categorical_crossentropy', 
    metrics=METRICS)
    print('-------------------------------------------')
    print(f'Fitting model on X_train, Y_train:')
    history = model.fit(X_train, Y_train, 
        batch_size=batch_size, 
        epochs=num_epochs,
        verbose=verbose)

    print(model.summary())

    scores = model.evaluate(X_test, Y_test, verbose=1)
    print(scores)
    print(model.metrics_names)
    zipped = list(zip(model.metrics_names, scores))
    scores_per_fold.append(zipped)
    
    print(f"Score for fold {fold_no}: {[zips for zips in zipped]}.")

    fold_no += 1


## Averaging metrics over 5 folds

In [24]:
total_scores = {}
for idx, list_of_scores in enumerate(scores_per_fold): 
    temp_scores = {scores[0]: scores[1] for scores in list_of_scores}
    total_scores[idx] = temp_scores


In [44]:
total_scores # note: the numbers here are inaccurate. Only the categorical accuracy should be used as is, since tp,fp,tn,fn do not work with multiclass labels

{0: {'loss': 0.20712690036973835,
  'tp': 1222.0,
  'fp': 53.0,
  'tn': 6427.0,
  'fn': 74.0,
  'categorical_accuracy': 0.94753087,
  'precision': 0.95843136,
  'recall': 0.94290125,
  'auc': 0.9954687,
  'prc': 0.9836373},
 1: {'loss': 0.21576449790118654,
  'tp': 1212.0,
  'fp': 62.0,
  'tn': 6418.0,
  'fn': 84.0,
  'categorical_accuracy': 0.9421296,
  'precision': 0.95133436,
  'recall': 0.9351852,
  'auc': 0.9957455,
  'prc': 0.98160946},
 2: {'loss': 0.18227159820589017,
  'tp': 1237.0,
  'fp': 51.0,
  'tn': 6429.0,
  'fn': 59.0,
  'categorical_accuracy': 0.9583333,
  'precision': 0.96040374,
  'recall': 0.9544753,
  'auc': 0.9969951,
  'prc': 0.9883691},
 3: {'loss': 0.2038687332360833,
  'tp': 1233.0,
  'fp': 53.0,
  'tn': 6427.0,
  'fn': 63.0,
  'categorical_accuracy': 0.9537037,
  'precision': 0.95878696,
  'recall': 0.9513889,
  'auc': 0.99491996,
  'prc': 0.9844293},
 4: {'loss': 0.19386949666120387,
  'tp': 1222.0,
  'fp': 53.0,
  'tn': 6427.0,
  'fn': 74.0,
  'categorical_

In [42]:
avg_scores = []
for col in ['loss', 'tp', 'fp', 'tn', 'fn', 'categorical_accuracy', 'precision', 'recall', 'auc', 'prc']:
    avg_val = 0
    for i in range(5):
        avg_val += total_scores[i][col]
    avg_scores.append((col, avg_val/5))

In [45]:
avg_scores

[('loss', 0.20058024527482043),
 ('tp', 1225.2),
 ('fp', 54.4),
 ('tn', 6425.6),
 ('fn', 70.8),
 ('categorical_accuracy', 0.949999988079071),
 ('precision', 0.9574775576591492),
 ('recall', 0.9453703761100769),
 ('auc', 0.9956256985664368),
 ('prc', 0.9842562913894654)]

The 1D convolution network performs very well on unseen data, reaching state-of-the-art performance. However, we point out that such performance is only possible in an academic environment where we have recorded labels for distinct damage states, with rigorous definitions for each damage state and a large amount of data in each. Such environments are not to be expected for real-world use-cases such as aircraft and vehicle maintenance, bridges etc. We show the performance to demonstrate what is _theoretically_ possible, and to note one possible way of leveraging multiple sensor data: simply use them all for training, without combining them in any form! In a sense, having 24 sensors worth of data (offset from each other spacially) is a useful way to replicate the successes of data augmentation in computer vision to the one-dimensional time series case.